# MERGE brickset WITH kaggle

Workflow: 3   

Goal: Merge the filtered brickset set data with kaggle inventory data (for number of parts, colors, etc).

Result: Outputs the file ```merge_full.csv```.

Author: Sorin Luca

In [1]:
import os
import pandas as pd

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## import brickset set data

In [3]:
FILTER_FILE = '../data/brickset_set_filter.csv'
INVENTORIES_FILE = '../data/kaggle_inventories.csv'
INVENTORYPARTS_FILE = '../data/kaggle_inventory_parts.csv'
COLORS_FILE = '../data/kaggle_colors.csv'
PARTS_FILE = '../data/kaggle_parts.csv'
PARTCATEGORIES_FILE = '../data/kaggle_part_categories.csv'

In [4]:
brickset = pd.read_csv(FILTER_FILE)
brickset[["set_no", "piece_cnt"]].head(3)

,set_no,piece_cnt
0,858-1,242.0
1,1591-1,40.0
2,1592-1,471.0


In [5]:
brickset.shape

(6181, 16)

## merge inventory_id as new column

In [6]:
kaggle_inventories = pd.read_csv(INVENTORIES_FILE)

kaggle_inventories.head(3)

,id,version,set_num
0,1,1,7922-1
1,3,1,3931-1
2,4,1,6942-1


In [7]:
## select needed columns
kaggle_inventories = kaggle_inventories[["set_num", "id"]]

## rename columns for merging
kaggle_inventories.columns = ["set_no", "inventory_id"]

kaggle_inventories.head(3)

,set_no,inventory_id
0,7922-1,1
1,3931-1,3
2,6942-1,4


In [8]:
brickset = pd.merge(brickset, kaggle_inventories, how="left")

## count missing entries
print(F'missing inventory_id for {brickset["inventory_id"].isna().sum()} entries')

brickset[["set_no", "piece_cnt", "inventory_id"]].head(3)

missing inventory_id for 663 entries


,set_no,piece_cnt,inventory_id
0,858-1,242.0,4886.0
1,1591-1,40.0,11940.0
2,1592-1,471.0,14106.0


In [9]:
## integer columns get cast to a float if NaNs are introduced
## some sets do not have an inventory_id

## delete NANs from "inventory_id"
brickset.dropna(subset=["inventory_id"], inplace=True)

## reset "inventory_id" to integers
brickset["inventory_id"] = brickset["inventory_id"].astype("int")

brickset[["set_no", "piece_cnt", "inventory_id"]].head(3)

,set_no,piece_cnt,inventory_id
0,858-1,242.0,4886
1,1591-1,40.0,11940
2,1592-1,471.0,14106


## process inventory_parts + colors

In [10]:
kaggle_inventory_parts = pd.read_csv(INVENTORYPARTS_FILE)

kaggle_inventory_parts.head(3)

,inventory_id,part_num,color_id,quantity,is_spare
0,1,48379c01,72,1,f
1,1,48395,7,1,f
2,1,mcsport6,25,1,f


In [11]:
kaggle_inventory_parts["is_spare"].value_counts()

f    550756
t     29495
Name: is_spare, dtype: int64

In [12]:
## transforming 'is_spare' to boolean values
kaggle_inventory_parts["is_spare"] = kaggle_inventory_parts["is_spare"].map({"f": 0, "t": 1})
kaggle_inventory_parts["spare_quantity"] = kaggle_inventory_parts["quantity"] * kaggle_inventory_parts["is_spare"]
kaggle_inventory_parts["is_spare"].value_counts()

0    550756
1     29495
Name: is_spare, dtype: int64

In [13]:
kaggle_colors = pd.read_csv(COLORS_FILE)

kaggle_colors.head(3)

,id,name,rgb,is_trans
0,-1,Unknown,0033B2,f
1,0,Black,05131D,f
2,1,Blue,0055BF,f


In [14]:
## subset columns
kaggle_colors = kaggle_colors[["id", "name"]]

## change column names for merge
kaggle_colors.columns = ["color_id", "color_name"]

kaggle_colors.head(3)

,color_id,color_name
0,-1,Unknown
1,0,Black
2,1,Blue


In [15]:
## merge
kaggle_parts = pd.merge(kaggle_inventory_parts, kaggle_colors, how="left")

kaggle_parts.head(3)

,inventory_id,part_num,color_id,quantity,is_spare,spare_quantity,color_name
0,1,48379c01,72,1,0,0,Dark Bluish Gray
1,1,48395,7,1,0,0,Light Gray
2,1,mcsport6,25,1,0,0,Orange


## merge unique and total part metrics

In [16]:
kaggle_number_parts = kaggle_parts[["inventory_id", "quantity"]].groupby(by="inventory_id").agg(['count', 'sum'])
kaggle_number_parts["number_spare_parts"] = kaggle_parts[["inventory_id", "is_spare"]].groupby(by="inventory_id").sum()
kaggle_number_parts["total_spare_parts"] = kaggle_parts[["inventory_id", "spare_quantity"]].groupby(by="inventory_id").sum()
kaggle_number_parts.reset_index(inplace=True)

## rename columns for merging
kaggle_number_parts.columns = ["inventory_id", "number_parts", "total_parts", "number_spare_parts", "total_spare_parts"]

kaggle_number_parts.head(3)

,inventory_id,number_parts,total_parts,number_spare_parts,total_spare_parts
0,1,4,4,0,0
1,3,29,46,3,3
2,4,16,21,1,1


In [17]:
brickset = pd.merge(brickset, kaggle_number_parts, how="left")

## count missing entries
print(F'missing number_parts for {brickset["number_parts"].isna().sum()} entries')

brickset[["inventory_id", "number_parts", "total_parts", "number_spare_parts", "total_spare_parts"]].head(3)

missing number_parts for 29 entries


,inventory_id,number_parts,total_parts,number_spare_parts,total_spare_parts
0,4886,46.0,242.0,0.0,0.0
1,11940,19.0,40.0,0.0,0.0
2,14106,133.0,495.0,0.0,0.0


In [18]:
## delete NANs from "number_parts"
brickset.dropna(subset=["number_parts"], inplace=True)

brickset[["set_no", "inventory_id", "number_parts"]].head(3)

,set_no,inventory_id,number_parts
0,858-1,4886,46.0
1,1591-1,11940,19.0
2,1592-1,14106,133.0


## throw away sets with no repeat pieces

In [19]:
filter = brickset["number_parts"] != brickset["total_parts"]
brickset = brickset[filter]
brickset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5303 entries, 0 to 5523
Data columns (total 21 columns):
set_no                5303 non-null object
name                  5303 non-null object
price_store           4178 non-null float64
price_new             5003 non-null float64
price_used            4972 non-null float64
rating_value          4560 non-null float64
rating_votes          4560 non-null float64
theme_group           5303 non-null object
theme                 5303 non-null object
subtheme              3915 non-null object
main_tag              0 non-null float64
year                  5303 non-null float64
volume                2143 non-null float64
weight                2154 non-null float64
piece_cnt             5296 non-null float64
minifig_cnt           5303 non-null float64
inventory_id          5303 non-null int32
number_parts          5303 non-null float64
total_parts           5303 non-null float64
number_spare_parts    5303 non-null float64
total_spare_parts     5

## merge number_of_colors

In [20]:
kaggle_number_colors = kaggle_parts[["inventory_id", "color_name"]].groupby(by="inventory_id")["color_name"].nunique()
kaggle_number_colors = kaggle_number_colors.reset_index()

## rename columns for merging
kaggle_number_colors.columns = ["inventory_id", "number_colors"]

kaggle_number_colors.head(6)

,inventory_id,number_colors
0,1,4
1,3,12
2,4,5
3,15,1
4,16,2
5,17,5


In [21]:
brickset = pd.merge(brickset, kaggle_number_colors, how="left")

## count missing entries
print(F'missing number_parts for {brickset["number_parts"].isna().sum()} entries')

brickset[["set_no", "inventory_id", "number_parts", "total_parts", "number_colors"]].head(3)

missing number_parts for 0 entries


,set_no,inventory_id,number_parts,total_parts,number_colors
0,858-1,4886,46.0,242.0,5
1,1591-1,11940,19.0,40.0,4
2,1592-1,14106,133.0,495.0,10


## merge colors

In [22]:
kaggle_parts.head(3)

,inventory_id,part_num,color_id,quantity,is_spare,spare_quantity,color_name
0,1,48379c01,72,1,0,0,Dark Bluish Gray
1,1,48395,7,1,0,0,Light Gray
2,1,mcsport6,25,1,0,0,Orange


In [23]:
kaggle_part_colors = kaggle_parts.groupby(["inventory_id", "color_name"])[["quantity"]].sum()
kaggle_part_colors = kaggle_part_colors.reset_index()

kaggle_part_colors = kaggle_part_colors.groupby(["inventory_id"]).apply(lambda row: row.nlargest(2, 'quantity'))

kaggle_part_colors = kaggle_part_colors.drop(columns="inventory_id")

kaggle_part_colors = kaggle_part_colors.reset_index()
kaggle_part_colors.head(10)

,inventory_id,level_1,color_name,quantity
0,1,0,Black,1
1,1,1,Dark Bluish Gray,1
2,3,6,Bright Pink,10
3,3,9,Lime,10
4,4,20,Trans-Neon Green,7
5,4,17,Blue,5
6,15,21,Green,2
7,16,22,Red,8
8,16,23,White,2
9,17,24,Black,15


In [24]:
for row in brickset.itertuples():
    color_filter = kaggle_part_colors["inventory_id"] == row.inventory_id
    brickset.loc[row.Index, "1st_color"] = (kaggle_part_colors[color_filter].iloc[0].loc["color_name"])
    brickset.loc[row.Index, "2nd_color"] = (kaggle_part_colors[color_filter].iloc[-1].loc["color_name"])

brickset[["set_no", "inventory_id", "number_parts", "total_parts", "number_colors", "1st_color", "2nd_color"]].head(3)

,set_no,inventory_id,number_parts,total_parts,number_colors,1st_color,2nd_color
0,858-1,4886,46.0,242.0,5,Blue,Light Gray
1,1591-1,11940,19.0,40.0,4,White,Blue
2,1592-1,14106,133.0,495.0,10,Black,Yellow


## merge unique and total parts metrics by part category id

In [25]:
kaggle_parts.head()

,inventory_id,part_num,color_id,quantity,is_spare,spare_quantity,color_name
0,1,48379c01,72,1,0,0,Dark Bluish Gray
1,1,48395,7,1,0,0,Light Gray
2,1,mcsport6,25,1,0,0,Orange
3,1,paddle,0,1,0,0,Black
4,3,11816pr0005,78,1,0,0,Light Flesh


In [26]:
kaggle_parts_names = pd.read_csv(PARTS_FILE)
kaggle_parts_names.head()

,part_num,name,part_cat_id
0,0687b1,Set 0687 Activity Booklet 1,17
1,0901,Baseplate 16 x 30 with Set 080 Yellow House Print,1
2,0902,Baseplate 16 x 24 with Set 080 Small White Hou...,1
3,0903,Baseplate 16 x 24 with Set 080 Red House Print,1
4,0904,Baseplate 16 x 24 with Set 080 Large White Hou...,1


In [27]:
kaggle_part_categories = pd.read_csv(PARTCATEGORIES_FILE)
kaggle_part_categories.head()

,id,name
0,1,Baseplates
1,2,Bricks Printed
2,3,Bricks Sloped
3,4,"Duplo, Quatro and Primo"
4,5,Bricks Special


In [28]:
kaggle_parts_names = pd.merge(kaggle_parts_names, kaggle_part_categories,
                              how="left", left_on="part_cat_id", right_on="id",
                             suffixes=("_part","_category"))
kaggle_parts_names.head()

,part_num,name_part,part_cat_id,id,name_category
0,0687b1,Set 0687 Activity Booklet 1,17,17,Non-LEGO
1,0901,Baseplate 16 x 30 with Set 080 Yellow House Print,1,1,Baseplates
2,0902,Baseplate 16 x 24 with Set 080 Small White Hou...,1,1,Baseplates
3,0903,Baseplate 16 x 24 with Set 080 Red House Print,1,1,Baseplates
4,0904,Baseplate 16 x 24 with Set 080 Large White Hou...,1,1,Baseplates


In [29]:
kaggle_parts = pd.merge(kaggle_parts, kaggle_parts_names[["part_num", "name_category"]], how="left",
                        on="part_num")
kaggle_parts.head()

,inventory_id,part_num,color_id,quantity,is_spare,spare_quantity,color_name,name_category
0,1,48379c01,72,1,0,0,Dark Bluish Gray,Minifig Accessories
1,1,48395,7,1,0,0,Light Gray,Minifig Accessories
2,1,mcsport6,25,1,0,0,Orange,Minifigs
3,1,paddle,0,1,0,0,Black,Minifig Accessories
4,3,11816pr0005,78,1,0,0,Light Flesh,Minifigs


In [30]:
kaggle_categories = kaggle_parts[["inventory_id", "quantity", "name_category"]].copy()
kaggle_categories.head()

,inventory_id,quantity,name_category
0,1,1,Minifig Accessories
1,1,1,Minifig Accessories
2,1,1,Minifigs
3,1,1,Minifig Accessories
4,3,1,Minifigs


In [31]:
# generating column names
categories_raw = list(kaggle_parts["name_category"].unique())
categories = []
for cat in categories_raw:
    temp = str(cat).lower()
    temp = temp.replace(" (duplo, fabuland, etc)","")
    temp = temp.replace(", ","_")
    temp = temp.replace(" ","_")
    categories.append(temp)
categories_cnt_prts = [cat+"_cnt_prts" for cat in categories]
categories_tot_prts = [cat+"_tot_prts" for cat in categories]

In [32]:
# generating a dictionary for iterating through column creation
category_cnt_dict = dict(zip(categories_raw, categories_cnt_prts))
category_tot_dict = dict(zip(categories_raw, categories_tot_prts))

In [33]:
# converting grouby columns to category for improved use in groupby
kaggle_categories["inventory_id"] = kaggle_categories["inventory_id"].astype('category')
kaggle_categories["name_category"] = kaggle_categories["name_category"].astype('category')

In [34]:
# generating the count for each combination of category and set
kaggle_category_prt_cnt = kaggle_categories.groupby(["inventory_id", "name_category"]).size().unstack(fill_value=0)

In [35]:
# reshaping the dataframe
kaggle_category_prt_cnt.columns = [category_cnt_dict[col] for col in kaggle_category_prt_cnt.columns]
kaggle_category_prt_cnt.reset_index(inplace=True)
kaggle_category_prt_cnt = kaggle_category_prt_cnt.astype("int64")

In [36]:
kaggle_category_prt_cnt[["inventory_id", "plates_cnt_prts"]].head()

,inventory_id,plates_cnt_prts
0,1,0
1,3,4
2,4,1
3,15,0
4,16,0


In [37]:
# generating the sum of the quantity for each combination of category and set
kaggle_category_prt_tot = kaggle_categories.groupby(["inventory_id",
                                                     "name_category"]).agg(lambda x: 
                                                                           kaggle_categories.iloc[x.index,1].sum()).unstack()

In [38]:
# reshaping the dataframe
kaggle_category_prt_tot.columns = [category_tot_dict[col[1]] for col in kaggle_category_prt_tot.columns]
kaggle_category_prt_tot.reset_index(inplace=True)
kaggle_category_prt_tot["inventory_id"] = kaggle_category_prt_tot["inventory_id"].astype("int")
kaggle_category_prt_tot.fillna(value=0, axis=0, inplace=True)
kaggle_category_prt_tot = kaggle_category_prt_tot.astype("int64")

In [39]:
kaggle_category_prt_tot[["inventory_id", "plates_tot_prts"]].head()

,inventory_id,plates_tot_prts
0,1,0
1,3,5
2,4,1
3,15,0
4,16,0


In [40]:
# merging into brickset
brickset = pd.merge(brickset, kaggle_category_prt_cnt, how="left", left_on="inventory_id", right_on="inventory_id")
brickset = pd.merge(brickset, kaggle_category_prt_tot, how="left", left_on="inventory_id", right_on="inventory_id")

## clean up final columns

In [41]:
brickset.drop(columns=["inventory_id", "main_tag"], inplace=True)

In [42]:
to_int_list =["year", "minifig_cnt", "number_parts",
              "total_parts", "number_spare_parts", "total_spare_parts"]
brickset[to_int_list] = brickset[to_int_list].astype("int64")

In [43]:
brickset.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5303 entries, 0 to 5302
Data columns (total 136 columns):
set_no                                              5303 non-null object
name                                                5303 non-null object
price_store                                         4178 non-null float64
price_new                                           5003 non-null float64
price_used                                          4972 non-null float64
rating_value                                        4560 non-null float64
rating_votes                                        4560 non-null float64
theme_group                                         5303 non-null object
theme                                               5303 non-null object
subtheme                                            3915 non-null object
year                                                5303 non-null int64
volume                                              2143 non-null float64
weight     

## export to csv

In [44]:
save_path = r'../data/merge_full.csv'
brickset.to_csv(save_path, index=False)
f'{round(os.path.getsize(save_path) /1e6, 2)} mb'

'1.93 mb'